In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.config import list_physical_devices
from tensorflow.image import ResizeMethod
from tensorflow.image import resize_with_pad

import utils

In [2]:
trainX, trainY, testX, testY = utils.load_dataset()

In [3]:
def ensemble_dl():
    inputs = keras.Input(shape=(32,32,3))

    all_models = utils.get_models('models')
    all_model_output = []

    counter = 0
    for m in all_models:
        m.trainable = False
        all_model_output.append(m(inputs))
        print(str(counter) + ' done')
        counter += 1
    
    x = layers.concatenate(all_model_output)

    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    outputs = layers.Dense(10, activation='softmax')(x)
    
    ensemble_model = keras.Model(inputs=inputs, outputs=outputs, name='ensemble_dl')
    ensemble_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy', keras.metrics.AUC()])
    return ensemble_model

In [4]:
model = ensemble_dl()
model.summary()

0 done
1 done
2 done
3 done
Model: "ensemble_dl"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
custom_2 (Functional)           (None, 10)           924106      input_1[0][0]                    
__________________________________________________________________________________________________
custom_1 (Functional)           (None, 10)           2524938     input_1[0][0]                    
__________________________________________________________________________________________________
transfer_densenet (Functional)  (None, 10)           7347338     input_1[0][0]                    
____________________________________________________________

In [6]:
callback = keras.callbacks.ModelCheckpoint('models/ensemble_dl', monitor='val_loss', save_best_only=True)

In [7]:
history = model.fit(x=trainX, y=trainY, batch_size=512, epochs=50, callbacks=callback, validation_data=(testX, testY))

Epoch 1/50
98/98 [==============================] - 647s 5s/step - loss: 0.7966 - categorical_accuracy: 0.7696 - auc: 0.9438 - val_loss: 0.9272 - val_categorical_accuracy: 0.8340 - val_auc: 0.9761
INFO:tensorflow:Assets written to: models/ensemble_dl/assets
Epoch 2/50
98/98 [==============================] - 21s 213ms/step - loss: 0.2182 - categorical_accuracy: 0.9445 - auc: 0.9939 - val_loss: 0.6561 - val_categorical_accuracy: 0.8397 - val_auc: 0.9811
INFO:tensorflow:Assets written to: models/ensemble_dl/assets
Epoch 3/50
98/98 [==============================] - 21s 214ms/step - loss: 0.1958 - categorical_accuracy: 0.9478 - auc: 0.9952 - val_loss: 0.4946 - val_categorical_accuracy: 0.8537 - val_auc: 0.9854
INFO:tensorflow:Assets written to: models/ensemble_dl/assets
Epoch 4/50
98/98 [==============================] - 21s 214ms/step - loss: 0.1919 - categorical_accuracy: 0.9476 - auc: 0.9953 - val_loss: 0.4720 - val_categorical_accuracy: 0.8535 - val_auc: 0.9843
INFO:tensorflow:Assets 

In [8]:
best_model = keras.models.load_model('models/ensemble_dl')

In [9]:
best_model.evaluate(x=testX, y=testY)

313/313 [==============================] - 513s 829ms/step - loss: 0.4720 - categorical_accuracy: 0.8536 - auc: 0.9843


[0.4720035195350647, 0.853600025177002, 0.9843108057975769]